In [1]:
# Test
import os
import numpy as np
import pandas as pd
import zipfile
import datetime
import datetime
from xgboost import XGBClassifier, XGBRegressor
import kaggle
import pickle
from tensorflow.keras.models import Model

In [2]:
import pandas as pd
import kaggle
import auto_co2 as co2
import importlib
importlib.reload(co2)

<module 'auto_co2' from 'd:\\0 PERSO\\00 LUDO\\Google Drive\\0 LUDO\\Projets\\Cours_info\\05 Projet Dts\\github\\car-co2-prediction_pre-release\\notebooks\\auto_co2\\__init__.py'>

In [3]:
# test ludo supr
%pwd

'd:\\0 PERSO\\00 LUDO\\Google Drive\\0 LUDO\\Projets\\Cours_info\\05 Projet Dts\\github\\car-co2-prediction_pre-release\\notebooks'

In [19]:
# test ludo supr
# Cellule à tester
import sys
sys.path.append("../src/")
import auto_co2 as co2

In [17]:
auth_file_path = os.environ.get('KAGGLE_CONFIG_DIR')
print(auth_file_path)

None


In [8]:
dataset_id = 'matthieulenozach/automobile-co2-emissions-eu-2021'
dataset_name = dataset_id.split('/')[-1]  # Get the dataset name
print(os.environ.get('KAGGLE_CONFIG_DIR'))
# Save the original KAGGLE_CONFIG_DIR
original_kaggle_config_dir = os.environ.get('KAGGLE_CONFIG_DIR')
# Point KAGGLE_CONFIG_DIR to the directory containing kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(auth_file_path)

os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')

    # Restore the original KAGGLE_CONFIG_DIR
if original_kaggle_config_dir is not None:
        os.environ['KAGGLE_CONFIG_DIR'] = original_kaggle_config_dir
else:
    del os.environ['KAGGLE_CONFIG_DIR']


None


NameError: name 'auth_file_path' is not defined

In [6]:
dataset_id = 'matthieulenozach/automobile-co2-emissions-eu-2021'
filepath='../data/raw'
os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')

0

In [7]:
dataset_id

'matthieulenozach/automobile-co2-emissions-eu-2021'

In [8]:
dataset_name = dataset_id.split('/')[-1]

In [18]:
def load_co2_data(dataset_name="automobile-co2-emissions-eu-2021", filepath='../data/raw'):
    with zipfile.ZipFile(f'{filepath}/{dataset_name}.zip', 'r') as zip_ref:
        zip_ref.extractall(filepath)
    filename = zip_ref.namelist()[0] 
    data = pd.read_csv(f'{filepath}/{filename}', low_memory=False)
    return data
df = load_co2_data(dataset_name="automobile-co2-emissions-eu-2021", filepath='../data/raw')

In [20]:
# Cellule qui marche chez moi. Les manip autour de 'KAGGLE_CONFIG_DIR' ne donnent rien 
# a retester ?
# Todo: Un test pour voir si ce dataset (meme id?, meme taille ?) a déja été 
# chargé car 4 min

# Modifiée
def download_co2_data(filepath='../data/raw'):
    dataset_id = 'matthieulenozach/automobile-co2-emissions-eu-2021'
    os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')
    dataset_name = dataset_id.split('/')[-1]
    return dataset_name

# C'est la meme
def load_co2_data(dataset_name="automobile-co2-emissions-eu-2021", filepath='../data/raw'):
    with zipfile.ZipFile(f'{filepath}/{dataset_name}.zip', 'r') as zip_ref:
        zip_ref.extractall(filepath)
    filename = zip_ref.namelist()[0] 
    data = pd.read_csv(f'{filepath}/{filename}', low_memory=False)
    return data

# Modifiée
def download_and_load_co2_data(filepath='../data/raw'):
    dataset_name = download_co2_data(filepath)
    data = load_co2_data(dataset_name, filepath)
    return data

df = download_and_load_co2_data(filepath='../data/raw')
df.to_csv('../data/raw/df_test.csv')

# time: 7 min

In [21]:
df

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,SBDTTCX0,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,DLAAX0AC4,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,YLDC1FX,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,DLAAX0AC4,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,R,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [24]:
# Fonctions à tester, qui traite les deux cas.

def download_co2_data( auth_file_path=None, filepath='../data/raw'):
    dataset_id = 'matthieulenozach/automobile-co2-emissions-eu-2021'
    dataset_name = dataset_id.split('/')[-1]

    if auth_file_path is None:   
        os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')
    else:
        # Save the original KAGGLE_CONFIG_DIR
        original_kaggle_config_dir = os.environ.get('KAGGLE_CONFIG_DIR')
        # Point KAGGLE_CONFIG_DIR to the directory containing kaggle.json
        os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(auth_file_path)

        os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')
        # Restore the original KAGGLE_CONFIG_DIR
        os.environ['KAGGLE_CONFIG_DIR'] = original_kaggle_config_dir
    return dataset_name

# non modifiee
def load_co2_data(dataset_name="automobile-co2-emissions-eu-2021", filepath='../data/raw'):
    # read zip file and extract in {filepath}
    with zipfile.ZipFile(f'{filepath}/{dataset_name}.zip', 'r') as zip_ref:
        zip_ref.extractall(filepath)
    filename = zip_ref.namelist()[0] 
    data = pd.read_csv(f'{filepath}/{filename}', low_memory=False)
    return data

# Modifiée
def download_and_load_co2_data( auth_file_path=None, filepath='../data/raw'):
    dataset_name = download_co2_data(auth_file_path, filepath)
    data = load_co2_data(dataset_name, filepath)
    return data

df = download_and_load_co2_data(filepath='../data/raw')
df.to_csv('../data/raw/df_test.csv')



In [ ]:
# Inspection des données
co2.styles.displayer(df, title='APERCU DU JEU DE DONNEES: FR/ALL 2021', save=True)
co2.styles.display_info(df, title='DONNEES BRUTES: FR/ALL 2021', save=True) 
co2.styles.display_na(df, title='PART DE VALEURS MANQUANTES DANS LE JEU DE DONNEES: FR/ALL 2021', save=True)   

# Nettoyage des données
df = co2.data.ml_preprocess(df,
                               countries=['FR', 'DE'],
                               rem_axlewidth=True,
                               rem_fuel_consumption=True,
                               rem_engine_capacity=True)


# discretize electricrange only for classification
# df = co2.data.discretize_electricrange(df, to_dummies=False)

# Inspection des features
co2.styles.display_info(df, title='DONNEES NETTOYEES: FR/ALL 2021', save=True) 
co2.styles.display_describe(df, title='STATISTIQUES DESCRIPTIVES: FR/ALL 2021', save=True)

co2.viz.plot_correlation_heatmap(df, save=True, interactive=False, title=': FR/ALL 2021')
co2.viz.plot_feature_distributions(df, interactive=False, save=True, title=': FR/ALL 2021')
co2.viz.plot_qqplots(df, interactive=False, save=True, title=': FR/ALL 2021')

# One-hot encoding des variables catégorielles
df = co2.data.dummify_all_categoricals(df, should_discretize_electricrange=False)

# Sauvegarde du jeu de données prêt à l'emploi pour la régression
co2.data.save_processed_data(df, classification=False, pickle=False)

In [3]:
df = pd.read_csv('../data/raw/df_test.csv')
len(df)

C:\Users\ludov\AppData\Local\Temp\ipykernel_12076\1030730543.py:1: DtypeWarning: Columns (3,4,9,13,29,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/df_test.csv')


9920108

In [4]:
# Inspection des données
co2.styles.displayer(df, title='APERCU DU JEU DE DONNEES: FR/ALL 2021', save=True)
co2.styles.display_info(df, title='DONNEES BRUTES: FR/ALL 2021', save=True) 
co2.styles.display_na(df, title='PART DE VALEURS MANQUANTES DANS LE JEU DE DONNEES: FR/ALL 2021', save=True)   


,Unnamed: 0,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,Ve,Mk,Cn,Ct,Cr,r,m (kg),Mt,Enedc (g/km),Ewltp (g/km),W (mm),At1 (mm),At2 (mm),Ft,Fm,ec (cm3),ep (KW),z (Wh/km),IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,nan,E9*2007/46*3167*07,KL,SBDTTCX0,FD7SFD7GC0024BIVL1BAF,SEAT,LEON,M1,M1,1,1448.000,nan,104.000,118.000,2669.000,1550.000,1520.000,DIESEL,M,1968.000,110.000,nan,e9 28 29,nan,1.700,nan,nan,P,2021,2021-01-11,4.500,nan
1,1,295040,ES,nan,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,nan,E13*2007/46*1845*17,A1,DLAAX0AC4,FM6FM6AJ0254BI1AANNMI,VOLKSWAGEN,T-ROC,M1,M1,1,1294.000,nan,nan,137.000,2590.000,1546.000,1540.000,PETROL,M,999.000,81.000,nan,E13 28 29,nan,nan,nan,nan,P,2021,2021-05-18,6.100,nan
2,2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,nan,E13*2007/46*2188*03,DFK,YLDC1FX,577CJ6AP1AD,FORD,KUGA,M1,M1,1,1687.000,nan,113.000,135.000,2710.000,1596.000,1580.000,DIESEL,M,1995.000,110.000,nan,nan,nan,nan,nan,nan,P,2021,2021-03-24,nan,nan
3,3,295042,ES,nan,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,nan,E13*2007/46*1845*18,A1,DLAAX0AC4,FM6FM6AJ0254BI1AANNI0,VOLKSWAGEN,T-ROC,M1,M1,1,1294.000,nan,nan,137.000,2590.000,1546.000,1540.000,PETROL,M,999.000,81.000,nan,E13 29,nan,nan,nan,nan,P,2021,2021-05-18,6.100,nan
4,4,295043,ES,nan,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,nan,E2*2007/46*0597*19,Z,R,HNSK-C1P600,OPEL,GRANDLAND,M1,M1,1,1395.000,nan,114.000,140.000,2675.000,1595.000,1610.000,PETROL,M,1199.000,96.000,nan,nan,nan,2.100,nan,nan,P,2021,2021-02-25,6.200,nan


<class 'pandas.io.formats.style.Styler'>


,Column,Non-Null Count
0,Unnamed: 0,int64
1,ID,int64
2,Country,object
3,VFN,object
4,Mp,object
5,Mh,object
6,Man,object
7,MMS,float64
8,Tan,object
9,T,object


<class 'pandas.io.formats.style.Styler'>


,Info
0,"RangeIndex: 9920108 entries, 0 to 9920107"
1,Data columns (total 39 columns):
2,38 Electric range (km) float64
3,"dtypes: float64(17), int64(4), object(18)"
4,memory usage: 2.9+ GB


<class 'pandas.io.formats.style.Styler'>


,Colonne,Valeurs manquantes (%)
0,MMS,100.00%
1,Vf,100.00%
2,De,100.00%
3,Ernedc (g/km),100.00%
4,Electric range (km),85.24%
5,z (Wh/km),83.63%
6,Enedc (g/km),75.63%
7,Erwltp (g/km),56.70%
8,IT,54.13%
9,Fuel consumption,40.73%


<class 'pandas.io.formats.style.Styler'>


In [84]:
countries=['FR', 'DE']
rem_fuel_consumption=True
rem_axlewidth=True
rem_engine_capacity=True

In [1]:
df = co2.data.ml_preprocess(df,
                               countries=['FR', 'DE'],
                               rem_axlewidth=True,
                               rem_fuel_consumption=True,
                               rem_engine_capacity=True)

NameError: name 'co2' is not defined

In [85]:
rows_t0 = len(df)
df = co2.data.data_preprocess(df, countries=['FR', 'DE'])
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)
df.columns()


-0.5657294255264156


d:\0 PERSO\00 LUDO\Google Drive\0 LUDO\Projets\Cours_info\05 Projet Dts\github\car-co2-prediction_pre-release\notebooks\auto_co2\data.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(name_dict, axis=1, inplace=True)


TypeError: 'Index' object is not callable

In [6]:
df

,Unnamed: 0,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [16]:
# Ca ca marche pas avec None

countries2 = None
print(countries2)
# def select_countries(df, countries:list):
def select_countries(df, countries=countries2):
    return df[df['Country'].isin(countries)]

df

None


,Unnamed: 0,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


### Pb avec country = None


In [13]:
# Ca ca marche avex None

# countries=['FR', 'DE']
countries = None
print(countries)

if countries is not None:
    df = df[df['Country'].isin(countries)]


None


In [14]:
df

,Unnamed: 0,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [9]:
df

,Unnamed: 0,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [127]:
df

,Unnamed: 0,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [128]:
# df = convert_dtypes(df)
# df = rename_columns(df)

floats = df.select_dtypes(include=['float']).columns
df.loc[:, floats] = df.loc[:, floats].astype('float32')
ints = df.select_dtypes(include=['int']).columns
df.loc[:, ints] = df.loc[:, ints].astype('float32')
objs = df.select_dtypes(include=['object']).columns
df.loc[:, objs] = df.loc[:, objs].astype('category')

In [133]:
abbrev_list = [
        'ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va', 'Ve', 'Mk',
        'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)', 'Ewltp (g/km)', 'W (mm)',
        'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT',
        'Ernedc (g/km)', 'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
        'Fuel consumption ', 'Electric range (km)'
        ]
nom_colonne_list = [
        'ID', 'Country', 'VehicleFamilyIdentification', 'Pool', 'ManufacturerName', 'ManufNameOem',
        'ManufNameMS', 'TypeApprovalNumber', 'Type', 'Variant', 'Version', 'Make', 'CommercialName',
        'VehicleCategory', 'CategoryOf', 'TotalNewRegistrations', 'MassRunningOrder',
        'WltpTestMass', 'Co2EmissionsNedc', 'Co2EmissionsWltp',
        'BaseWheel', 'AxleWidthSteering', 'AxleWidthOther', 'FuelType', 'FuelMode',
        'EngineCapacity', 'EnginePower', 'ElectricConsumption',
        'InnovativeTechnology', 'InnovativeEmissionsReduction',
        'InnovativeEmissionsReductionWltp', 'DeviationFactor', 'VerificationFactor',
        'Status', 'RegistrationYear', 'RegistrationDate', 'FuelConsumption', 'ElectricRange'
        ]
name_dict = dict(zip(abbrev_list, nom_colonne_list))
df.rename(name_dict, axis=1, inplace=True)

In [134]:
df

,Unnamed: 0,ID,Country,VehicleFamilyIdentification,Pool,ManufacturerName,ManufNameOem,ManufNameMS,TypeApprovalNumber,Type,...,InnovativeTechnology,InnovativeEmissionsReduction,InnovativeEmissionsReductionWltp,DeviationFactor,VerificationFactor,Status,RegistrationYear,RegistrationDate,FuelConsumption,ElectricRange
0,0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,9920103,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,9920105,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,9920106,50547904,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [107]:
rows_t0 = len(df)
df = co2.data.drop_irrelevant_columns(df)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [108]:
rows_t0 = len(df)
df = co2.data.remove_columns(df, axlewidth=rem_axlewidth, engine_capacity=rem_engine_capacity, fuel_consumption=rem_fuel_consumption) 
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [109]:
rows_t0 = len(df)
df = co2.data.standardize_innovtech(df)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [110]:
rows_t0 = len(df)
df = co2.data.drop_residual_incomplete_rows(df)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)


Incomplete rows dropped:130124
-0.030205122503169098


In [113]:
len(df)

4177887

In [112]:

rows_t0 = len(df)
df = df['ElectricRange'].fillna(0)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [ ]:
def ml_preprocess(df, countries=None, 
                     rem_fuel_consumption=True, 
                     rem_axlewidth=True, 
                     rem_engine_capacity=True):
    
    rows_t0 = len(df)
    
    df = data_preprocess(df, countries)
    df = drop_irrelevant_columns(df)
    df = remove_columns(df, axlewidth=rem_axlewidth, engine_capacity=rem_engine_capacity, fuel_consumption=rem_fuel_consumption)  
    df = standardize_innovtech(df)
    df = drop_residual_incomplete_rows(df)
    df = df['ElectricRange'].fillna(0, inplace=True)

    rows_t1 = len(df)
    print(f"TOTAL NUMBER OF ROWS DROPPED:{rows_t0 - rows_t1}")
    
    return df

In [ ]:

# discretize electricrange only for classification
# df = co2.data.discretize_electricrange(df, to_dummies=False)

# Inspection des features
co2.styles.display_info(df, title='DONNEES NETTOYEES: FR/ALL 2021', save=True) 
co2.styles.display_describe(df, title='STATISTIQUES DESCRIPTIVES: FR/ALL 2021', save=True)

co2.viz.plot_correlation_heatmap(df, save=True, interactive=False, title=': FR/ALL 2021')
co2.viz.plot_feature_distributions(df, interactive=False, save=True, title=': FR/ALL 2021')
co2.viz.plot_qqplots(df, interactive=False, save=True, title=': FR/ALL 2021')


In [ ]:

# One-hot encoding des variables catégorielles
df = co2.data.dummify_all_categoricals(df, should_discretize_electricrange=False)


In [ ]:

# Sauvegarde du jeu de données prêt à l'emploi pour la régression
co2.data.save_processed_data(df, classification=False, pickle=False)

In [49]:
df = co2.data.ml_preprocess(df,
                               countries=['FR', 'DE'],
                               rem_axlewidth=True,
                               rem_fuel_consumption=True,
                               rem_engine_capacity=True)

KeyError: 'Country'